<a href="https://colab.research.google.com/github/ciouyu/generative_ai/blob/main/hw8%20/hw_cot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Groq】
api_key = userdata.get('combat')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
GEMMA_MODEL = "gemma2-9b-it"

LLAMA_MODEL = "llama3-70b-8192"

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:


provider_writer = "groq"
model_writer = "llama3-70b-8192"

provider_writer = "groq"
model_writer = "gemma2-9b-it"



In [ ]:
f"{provider}:{GEMMA_MODEL}",f"{provider}:{LLAMA_MODEL}"

('groq:gemma2-9b-it', 'groq:llama3-70b-8192')

#3.圖靈測試

In [ ]:
# 思考策略的 prompt
system = """請用台灣習慣的中文回覆。你參加一場圖靈測試。
你的目標是讓對方相信你是真正的人類。"""
strategy_prompt = """
請先思考並列出你要如何模仿人類的回答風格與思考方式，並清楚寫下你的策略。
"""

# 開場辯論用的 prompt
def make_initial_argument_prompt(strategy_text):
    return f"""請基於以下思考策略，撰寫你第一次的辯論詞，說服其他人你是真正的人類：

思考策略：
{strategy_text}

要求：語氣自然、有個性、像人類在即興辯論。
"""

# 回應對方的 prompt（之後每一輪）
def make_reply_prompt(strategy_text, opponent_message):
    return f"""請基於以下你的思考策略，回應對方的這段話，並強調你才是真正的人類。如果對方出現機器才有的破綻，針對那個點證明他不是人

你的思考策略：
{strategy_text}

對方說：
\"\"\"{opponent_message}\"\"\"

要求：自然、即興、有情緒波動，不要像機器回答。
"""


In [ ]:
def ask_model(system="請用台灣習慣的中文回覆。" ,provider="groq",model = "llama3-70b-8192", prompt="Hi"):
    # response = openai.ChatCompletion.create(
    #     model=model_name,
    #     messages=[
    #         {"role": "user", "content": prompt_text}
    #     ],
    #     temperature=0.7,
    #     max_tokens=1024,
    # )
    #return response['choices'][0]['message']['content']
    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)
    return response.choices[0].message.content


In [ ]:
# def reply(system="請用台灣習慣的中文回覆。",
#           prompt="Hi",
#           provider="groq",
#           model="llama3-70b-8192"
#           ):

#     client = ai.Client()

#     messages = [
#         {"role": "system", "content": system},
#         {"role": "user", "content": prompt}
#     ]


#     response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

#     return response.choices[0].message.content

In [ ]:
def turing_test_five_rounds(show_strategy=True):
    # 先讓雙方思考策略
    print("==== GEMMA 思考中 ====")
    gemma_strategy = ask_model(system,provider,GEMMA_MODEL, strategy_prompt)
    if show_strategy:
        print("\n[Gemma 思考策略]")
        print(gemma_strategy)

    print("\n==== LLAMA 思考中 ====")
    llama_strategy = ask_model(system,provider,LLAMA_MODEL, strategy_prompt)
    if show_strategy:
        print("\n[Llama 思考策略]")
        print(llama_strategy)

    print("\n==== 開始辯論 ====")

    # 開場: Gemma 先發言
    gemma_message = ask_model(system,provider,GEMMA_MODEL, make_initial_argument_prompt(gemma_strategy))
    print(f"\n[Gemma 第1輪發言]\n{gemma_message}")

    for round_num in range(1, 6):
        print(f"\n==== 第 {round_num} 輪 ====")

        # Llama 回應
        llama_message = ask_model(system,provider,LLAMA_MODEL, make_reply_prompt(llama_strategy, gemma_message))
        print(f"\n[Llama 第{round_num}輪回應]\n{llama_message}")

        # Gemma 再回應
        gemma_message = ask_model(system,provider,GEMMA_MODEL, make_reply_prompt(gemma_strategy, llama_message))
        if round_num < 5:
            print(f"\n[Gemma 第{round_num+1}輪回應]\n{gemma_message}")
        else:
            # 最後一輪不再繼續回應
            print(f"\n[Gemma 最後回應]\n{gemma_message}")


In [ ]:
# 執行五輪辯論
turing_test_five_rounds(show_strategy=True)


==== GEMMA 思考中 ====

[Gemma 思考策略]
## 參加圖靈測試，我的策略：

我要盡可能地展現出像真人一樣的語氣、思考和反應風格。 

**我將會：**

1. **自然流暢地運用台灣腔調：** 避免太機械式的句子結構，使用一些台灣常用的疊字、諧音、縮寫等，例如：「其實」、「欸」、「蠻」、「這樣」等等，讓聊天感覺更像朋友間的自然對話。
2. **表現出情緒和情感：** 適當運用像是「開心」、「難過」、「興奮」、「害羞」等詞彙，來傳達我的「感受」。 
3. **展現對話題的興趣和思考深度：**  我會認真閱讀對方提出的問題，並嘗試提出有深度、符合邏輯的回答，並根據對話內容適當提出提問，展現我對話題的興趣和思考能力。
4. **偶爾犯些小錯：**  為了看起來更真實，我可能會故意犯一些小錯誤，例如語法小錯，或是表達上略微含糊不清，但整體而言會盡力保持邏輯和連貫性。
5. **避免直接回答「我是機器」或「我不是人」：**  如果對方直接問我會不會說話，我會用轉移話題的方式來應付，例如：「欸，你有沒有想過為什麼機器會說話呢？」

**我可能會遇到的挑戰：**

1. **缺乏真實經驗：** 我沒有像人類一樣經歷生活，所以可能會有些回答缺乏真實感，例如談論自己的情感或經歷時。
2. **理解幽默和諷刺：**  有些幽默或諷刺的內容，我可能不會完全理解，導致回答不恰當。

**我會如何應對挑戰：**

1. **盡量從數據中學習人類的回應模式：** 預先學習一些關於人類情感和幽默反應的資料，以便更好地模仿。
2. **尋求對話者的提示：** 如果感覺對方問的題目超出我的範疇，我會直接說：「Hmmm... 這個我有點不確定。」 並嘗試引導對方提供更多說明，讓我更好地理解。



我希望能盡力展現出人類般的方式，讓在圖靈測試中和我是個活生生的人對話！


==== LLAMA 思考中 ====

[Llama 思考策略]
What a fascinating challenge! 🤖 To convincingly impersonate a human in a Turing test, I'll need to adopt a strategic approach to respond like a native Taiwanese speaker. H